# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.47,74,37,9.65,TF,1693043861
1,1,olonkinbyen,70.9221,-8.7187,6.47,97,41,8.00,SJ,1693043862
2,2,wailua homesteads,22.0669,-159.3780,26.44,82,40,7.72,US,1693043863
3,3,touros,-5.1989,-35.4608,25.47,79,59,3.80,BR,1693043759
4,4,grytviken,-54.2811,-36.5092,-5.08,78,85,9.71,GS,1693043865


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)


# Display the map 
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:

min_temp = city_data_df["Max Temp"] > 21
max_temp = city_data_df["Max Temp"] < 27
wind_speed = city_data_df["Wind Speed"] < 4.5


print(sum(min_temp))
print(sum(max_temp))
print(sum(wind_speed))


329
390
372


In [17]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_range_df = city_data_df[min_temp & max_temp & wind_speed ]
city_temp_range_df

# Drop any rows with null values
city_temp_df = city_temp_range_df.dropna()

# Display sample data
city_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,touros,-5.1989,-35.4608,25.47,79,59,3.80,BR,1693043759
20,20,puerto ayora,-0.7393,-90.3518,23.64,96,20,1.79,EC,1693043885
21,21,bonthe,7.5264,-12.5050,25.05,87,100,3.73,SL,1693043886
23,23,winnemucca,40.9730,-117.7357,22.03,46,0,3.09,US,1693043889
25,25,shimoda,34.6667,138.9500,26.72,84,76,3.88,JP,1693043891
...,...,...,...,...,...,...,...,...,...,...
543,543,toppenish,46.3774,-120.3087,22.15,57,0,2.05,US,1693044619
547,547,buchanan,5.8808,-10.0467,26.48,82,100,3.34,LR,1693044625
552,552,tamaqua,40.7973,-75.9694,21.79,95,75,0.45,US,1693044631
553,553,weihai,37.5017,122.1136,24.56,67,61,3.92,CN,1693044632


In [18]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_range_df = city_data_df[min_temp & max_temp & wind_speed]
city_temp_range_df

# Drop any rows with null values
clean_city_temp_range_df = city_temp_range_df.dropna()

# Display sample data
clean_city_temp_range_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,touros,-5.1989,-35.4608,25.47,79,59,3.80,BR,1693043759
20,20,puerto ayora,-0.7393,-90.3518,23.64,96,20,1.79,EC,1693043885
21,21,bonthe,7.5264,-12.5050,25.05,87,100,3.73,SL,1693043886
23,23,winnemucca,40.9730,-117.7357,22.03,46,0,3.09,US,1693043889
25,25,shimoda,34.6667,138.9500,26.72,84,76,3.88,JP,1693043891
...,...,...,...,...,...,...,...,...,...,...
543,543,toppenish,46.3774,-120.3087,22.15,57,0,2.05,US,1693044619
547,547,buchanan,5.8808,-10.0467,26.48,82,100,3.34,LR,1693044625
552,552,tamaqua,40.7973,-75.9694,21.79,95,75,0.45,US,1693044631
553,553,weihai,37.5017,122.1136,24.56,67,61,3.92,CN,1693044632


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df [["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(17)

,City,Country,Lat,Lng,Humidity,Hotel Name
3,touros,BR,-5.1989,-35.4608,79,
20,puerto ayora,EC,-0.7393,-90.3518,96,
21,bonthe,SL,7.5264,-12.5050,87,
23,winnemucca,US,40.9730,-117.7357,46,
25,shimoda,JP,34.6667,138.9500,84,
33,sinabang,ID,2.4803,96.3801,88,
37,uaua,BR,-9.8414,-39.4817,71,
45,kurchatov,RU,51.6601,35.6521,46,
51,kayenta,US,36.7278,-110.2546,53,
57,ust'-ilimsk,RU,58.0006,102.6619,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
touros - nearest hotel: Pousada Atlântico
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
bonthe - nearest hotel: No hotel found
winnemucca - nearest hotel: The Overland Hotel
shimoda - nearest hotel: ホテルウラガ
sinabang - nearest hotel: No hotel found
uaua - nearest hotel: Hotel Uauá
kurchatov - nearest hotel: Россия
kayenta - nearest hotel: Wetherill Inn
ust'-ilimsk - nearest hotel: Усть-Илимск
sao jose da coroa grande - nearest hotel: Hotel do Sol
mglin - nearest hotel: No hotel found
les cayes - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
arenapolis - nearest hotel: No hotel found
mazagao - nearest hotel: No hotel found
manokwari - nearest hotel: Hotel Arfak
jimma - nearest hotel: CITY VIEW Hotel
hawaiian paradise park - nearest hotel: No hotel found
markovo - nearest hotel: No hotel found
mahina - nearest hotel: Motu Martin
kapa'a - nearest hotel: Pono Kai Resort
aripuana - nearest hotel: No hotel found
pangoa - ne

,City,Country,Lat,Lng,Humidity,Hotel Name
3,touros,BR,-5.1989,-35.4608,79,Pousada Atlântico
20,puerto ayora,EC,-0.7393,-90.3518,96,Hostal La Mirada De Solitario George
21,bonthe,SL,7.5264,-12.5050,87,No hotel found
23,winnemucca,US,40.9730,-117.7357,46,The Overland Hotel
25,shimoda,JP,34.6667,138.9500,84,ホテルウラガ
...,...,...,...,...,...,...
543,toppenish,US,46.3774,-120.3087,57,Quality Inn & Suites
547,buchanan,LR,5.8808,-10.0467,82,Phillipmena's Guesthouse
552,tamaqua,US,40.7973,-75.9694,95,No hotel found
553,weihai,CN,37.5017,122.1136,67,Huakun


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)